In [1]:
import requests as reg
from bs4 import BeautifulSoup as soup

url_reg=reg.get("https://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
Content=url_reg.content

soup_page = soup(Content,"html.parser")

number_of_pages=soup_page.findAll("a",{"class":"Page"})[-1].text
print(number_of_pages,"number of pages were found")

3 number of pages were found


In [2]:
dataset=[]
base_url="http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for page in range(0,int(number_of_pages)*10,10):
    url_reg=reg.get(base_url+str(page)+".html")
    Content=url_reg.content
    soup_page=soup(Content,"html.parser")
    places=soup_page.find_all("div",{"class":"propertyRow"})
    for item in places:
        data={}
        data["Address"]=item.find_all("span",{"class","propAddressCollapse"})[0].text
        try:
            data["Locality"]=item.find_all("span",{"class","propAddressCollapse"})[1].text
        except:
            data["Locality"]=None
        data["Price"]=item.find("h4",{"class","propPrice"}).text.replace("\n","").replace(" ","")
        try:
            data["Beds"]=item.find("span",{"class","infoBed"}).find("b").text
        except:
            data["Beds"]=None
    
        try:
            data["Area"]=item.find("span",{"class","infoSqFt"}).find("b").text
        except:
            data["Area"]=None
    
        try:
            data["Full Baths"]=item.find("span",{"class","infoValueFullBath"}).find("b").text
        except:
            data["Full Baths"]=None

        try:
            data["Half Baths"]=item.find("span",{"class","infoValueHalfBath"}).find("b").text
        except:
            data["Half Baths"]=None
        for column_group in item.find_all("div",{"class":"columnGroup"}):
            for feature_group, feature_name in zip(column_group.find_all("span",{"class":"featureGroup"}),column_group.find_all("span",{"class":"featureName"})):
                if "Lot Size" in feature_group.text:
                    data["Lot Size"]=feature_name.text
        dataset.append(data)

In [3]:
import pandas
dataframe=pandas.DataFrame(dataset)
dataframe.to_csv("Output.csv")

In [4]:
dataframe.head()

,Address,Area,Beds,Full Baths,Half Baths,Locality,Lot Size,Price
0,0 Gateway,None,None,None,None,"Rock Springs, WY 82901",NaN,"$725,000"
1,1003 Winchester Blvd.,None,4,4,None,"Rock Springs, WY 82901",0.21 Acres,"$452,900"
2,600 Talladega,"3,154",5,3,None,"Rock Springs, WY 82901",NaN,"$396,900"
3,3239 Spearhead Way,"3,076",4,3,1,"Rock Springs, WY 82901","Under 1/2 Acre,","$389,900"
4,522 Emerald Street,"1,172",3,3,None,"Rock Springs, WY 82901","Under 1/2 Acre,","$254,000"
